In [13]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [43]:
#dataset1filepath = "/Users/adityagautam/Downloads/mini-project-1/datasets/train/train_emoticon.csv"
dfemoji = pd.read_csv("/Users/adityagautam/Downloads/mini-project-1/datasets/train/train_emoticon.csv")
dfcode = pd.read_csv("/Users/adityagautam/Downloads/mini-project-1/datasets/train/train_text_seq.csv")

In [20]:
dfemoji

,input_emoticon,label
0,😛🛐😻😑😣🙠🙯🚼😒🙼😑🙯😣,0
1,🛐😑😪😛🚼🙯😣🚅😑🙯😹😣🙼,0
2,😛🙯😑🚡😣🚼🛐🙲😣🙯🛑😑🙼,0
3,😛🚼🛐🙐😣🙯😑🙪😑🙼🛆😣🙯,1
4,🛐🚟🚼😛🙋😑😣🙯😹🙯😑😣🙼,1
...,...,...
7075,🙯😺😻😑😣🛐😛🚼🙼🙯😣😑🚠,1
7076,🛐🚼😅🙯😣🙹😛😑😑🙼😣🚍🙯,1
7077,🛜😑🙒😛😣🙯🛐🚼🙼🙬🙯😣😑,1
7078,😣😑😛🛋🛐🙯🚼🚪🙯😑🛀😣🙼,0


In [54]:
# Ensure dfemoji is a DataFrame and extract the last character (emoji) from 'input_emoticon'
dfemoji = pd.DataFrame({'last_emoji': dfemoji['input_emoticon'].str[-1]})

In [55]:
dfemoji

,last_emoji
0,😣
1,🙼
2,🙼
3,🙯
4,🙼
...,...
7075,🚠
7076,🙯
7077,😑
7078,🙼


In [44]:
dfcode = dfcode.drop('label', axis=1) 
dfcode

,input_str
0,0000154364642718159661428002624223132284159626...
1,0004641596369515436422262614110471596262476161...
2,0001543626215965999614422464135806142624051159...
3,0000154364224641238614262159689561596284351061...
4,0004641899422154362069015966142624761262159661...
...,...
7075,0000026267027181596614464154364222842626141596...
7076,0000464422155826261433491543615961596284614390...
7077,0003069915964309154366142624644222841795262614...
7078,0000614159615436198346426242237758262159694061...


In [25]:
import pandas as pd

# Function to find the longest common suffix in a list of strings
def find_common_suffix(strings):
    if not strings:
        return ""
    
    # Reverse the strings to compare from the end
    rev_strings = [s[::-1] for s in strings]
    common_suffix = []
    
    # Compare characters one by one from the end (actually reversed)
    for chars in zip(*rev_strings):
        if all(c == chars[0] for c in chars):
            common_suffix.append(chars[0])
        else:
            break
    
    # Reverse back the common suffix to its original order
    return ''.join(common_suffix)[::-1]

# Ensure both DataFrames have the same index, and columns are correctly named
assert len(dfemoji) == len(dfcode), "Mismatch in number of rows between dfemoji and dfcode."

# Group by the unique emojis in the 'last_emoji' column
emoji_groups = dfemoji.groupby('last_emoji')

# Prepare to store results
results = []

# Iterate through each emoji group
for emoji, group in emoji_groups:
    # Get the corresponding input strings (rows in dfcode that match the group rows)
    matched_rows = dfcode.loc[group.index, 'input_str']  # Assuming 'input_str' is the column name in dfcode
    
    # Convert the matching rows to a list of strings
    codes = matched_rows.tolist()
    
    # Find the common suffix in these codes
    common_suffix = find_common_suffix(codes)
    
    # Append the result to the list
    results.append({'emoji': emoji, 'code': common_suffix})

# Create a new DataFrame with the results
dfresult = pd.DataFrame(results)

# Display the result DataFrame
print(dfresult)

   emoji  code
0      😁  6659
1      😅  1558
2      😆  2075
3      😇  2078
4      😉  3104
..   ...   ...
86     🛘  1507
87     🛙   486
88     🛛  3559
89     🛟  2548
90     🛡  3070

[91 rows x 2 columns]


In [61]:
#dfresultlastcol = dfresult
dfresultlastcol

,emoji,code
0,😁,6659
1,😅,1558
2,😆,2075
3,😇,2078
4,😉,3104
...,...,...
86,🛘,1507
87,🛙,486
88,🛛,3559
89,🛟,2548


In [40]:
# Calculate the length of each string in the 'code' column
string_lengths = dfresultlastcol['code'].dropna().apply(len)

# Count the frequency of each string length
length_frequency = string_lengths.value_counts().sort_index()

# Display the frequency of string sizes
print(length_frequency)

3     5
4    86
Name: code, dtype: int64


In [48]:
import pandas as pd

# Assuming dfresult has columns 'emoji' and 'code', dfemoji has a column with a string of 13 emojis,
# and dfcode has 'input_str' (the corresponding ciphered number strings)

# Step 1: Create a dictionary mapping emojis to their corresponding code
emoji_to_code = dict(zip(dfresult['emoji'], dfresult['code']))

# Step 2: Define a function to generate the code string from the last 5 emojis
def generate_code_string_from_last_5_emojis(emoji_string):
    last_5_emojis = emoji_string[-5:]  # Get the last 5 emojis
    code_string = ''.join([emoji_to_code.get(emoji, '') for emoji in last_5_emojis])  # Get codes for each emoji
    return code_string

# Step 3: Apply the function to dfemoji to generate the code string for each row
dfemoji['code_string'] = dfemoji.iloc[:, 0].apply(generate_code_string_from_last_5_emojis)

# Step 4: Remove the generated code string from the corresponding row in dfcode
# We iterate through rows to ensure correct removal
for idx, code_str in enumerate(dfemoji['code_string']):
    if pd.notnull(dfcode.at[idx, 'input_str']) and code_str:  # Check if the input string is not NaN and code_str is not empty
        dfcode.at[idx, 'input_str'] = dfcode.at[idx, 'input_str'].replace(code_str, '', 1)  # Remove only the first occurrence

# Display the updated dfcode dataframe
dfemoji

,input_emoticon,label,code_string
0,😛🛐😻😑😣🙠🙯🚼😒🙼😑🙯😣,0,31322841596262614
1,🛐😑😪😛🚼🙯😣🚅😑🙯😹😣🙼,0,15962624761614284
2,😛🙯😑🚡😣🚼🛐🙲😣🙯🛑😑🙼,0,61426240511596284
3,😛🚼🛐🙐😣🙯😑🙪😑🙼🛆😣🙯,1,15962843510614262
4,🛐🚟🚼😛🙋😑😣🙯😹🙯😑😣🙼,1,47612621596614284
...,...,...,...
7075,🙯😺😻😑😣🛐😛🚼🙼🙯😣😑🚠,1,28426261415962414
7076,🛐🚼😅🙯😣🙹😛😑😑🙼😣🚍🙯,1,15962846143901262
7077,🛜😑🙒😛😣🙯🛐🚼🙼🙬🙯😣😑,1,28417952626141596
7078,😣😑😛🛋🛐🙯🚼🚪🙯😑🛀😣🙼,0,2621596940614284


In [49]:
# Create a new column 'code_string_length' to store the length of strings in 'code_string'
dfemoji['code_string_length'] = dfemoji['code_string'].apply(lambda x: len(x) if isinstance(x, str) else 0)

# Display the updated dfemoji dataframe
print(dfemoji)

     input_emoticon  label        code_string  code_string_length
0     😛🛐😻😑😣🙠🙯🚼😒🙼😑🙯😣      0  31322841596262614                  17
1     🛐😑😪😛🚼🙯😣🚅😑🙯😹😣🙼      0  15962624761614284                  17
2     😛🙯😑🚡😣🚼🛐🙲😣🙯🛑😑🙼      0  61426240511596284                  17
3     😛🚼🛐🙐😣🙯😑🙪😑🙼🛆😣🙯      1  15962843510614262                  17
4     🛐🚟🚼😛🙋😑😣🙯😹🙯😑😣🙼      1  47612621596614284                  17
...             ...    ...                ...                 ...
7075  🙯😺😻😑😣🛐😛🚼🙼🙯😣😑🚠      1  28426261415962414                  17
7076  🛐🚼😅🙯😣🙹😛😑😑🙼😣🚍🙯      1  15962846143901262                  17
7077  🛜😑🙒😛😣🙯🛐🚼🙼🙬🙯😣😑      1  28417952626141596                  17
7078  😣😑😛🛋🛐🙯🚼🚪🙯😑🛀😣🙼      0   2621596940614284                  16
7079  😛😣🚼😑🛐😹🙯🚭😑😣🚟🙯🙼      0  15966141899262284                  17

[7080 rows x 4 columns]


In [50]:
# Assuming dfemoji has a column 'code_string_length' and dfcode has the same number of rows
dfcode['code_string_length'] = dfemoji['code_string_length']

# Display the updated dfcode dataframe
print(dfcode)

                               input_str  code_string_length
0      000015436464271815966142800262422                  17
1      000464159636951543642226261411047                  17
2      000154362621596599961442246413580                  17
3      000015436422464123861426215968956                  17
4      000464189942215436206901596614262                  17
...                                  ...                 ...
7075   000002626702718159661446415436422                  17
7076   000046442215582626143349154361596                  17
7077   000306991596430915436614262464422                  17
7078  0000614159615436198346426242237758                  16
7079   000015436614422159646447612621923                  17

[7080 rows x 2 columns]


In [53]:
import pandas as pd

# Create dfcodeleft by removing the last x letters from input_str
dfcodeleft = dfcode.copy()  # Make a copy of dfcode to create dfcodeleft

# Define a function to truncate the string
def truncate_string(input_str, length):
    if isinstance(input_str, str) and isinstance(length, int):  # Ensure both are valid
        return input_str[:-length] if length <= len(input_str) else ''  # Truncate or return empty
    return input_str  # Return unchanged if conditions are not met

# Apply the truncation function
dfcodeleft['input_str'] = dfcodeleft.apply(lambda row: truncate_string(row['input_str'], row['code_string_length']), axis=1)

# Display the new dfcodeleft dataframe

dfcodeleft = dfcodeleft.drop('code_string_length', axis=1) 
dfcodeleft

,input_str
0,0000154364642718
1,0004641596369515
2,0001543626215965
3,0000154364224641
4,0004641899422154
...,...
7075,0000026267027181
7076,0000464422155826
7077,0003069915964309
7078,000061415961543619


In [58]:
dfemoji = pd.read_csv("/Users/adityagautam/Downloads/mini-project-1/datasets/train/train_emoticon.csv")
dfemoji = pd.DataFrame({'last_emoji': dfemoji['input_emoticon'].str[7]})
dfemoji

,last_emoji
0,🚼
1,🚅
2,🙲
3,🙪
4,🙯
...,...
7075,🚼
7076,😑
7077,🚼
7078,🚪


In [60]:
import pandas as pd

# Function to find the longest common suffix in a list of strings
def find_common_suffix(strings):
    if not strings:
        return ""
    
    # Reverse the strings to compare from the end
    rev_strings = [s[::-1] for s in strings]
    common_suffix = []
    
    # Compare characters one by one from the end (actually reversed)
    for chars in zip(*rev_strings):
        if all(c == chars[0] for c in chars):
            common_suffix.append(chars[0])
        else:
            break
    
    # Reverse back the common suffix to its original order
    return ''.join(common_suffix)[::-1]

# Ensure both DataFrames have the same index, and columns are correctly named
assert len(dfemoji) == len(dfcode), "Mismatch in number of rows between dfemoji and dfcode."

# Group by the unique emojis in the 'last_emoji' column
emoji_groups = dfemoji.groupby('last_emoji')

# Prepare to store results
results = []

# Iterate through each emoji group
for emoji, group in emoji_groups:
    # Get the corresponding input strings (rows in dfcode that match the group rows)
    matched_rows = dfcode.loc[group.index, 'input_str']  # Assuming 'input_str' is the column name in dfcode
    
    # Convert the matching rows to a list of strings
    codes = matched_rows.tolist()
    
    # Find the common suffix in these codes
    common_suffix = find_common_suffix(codes)
    
    # Append the result to the list
    results.append({'emoji': emoji, 'code': common_suffix})

# Create a new DataFrame with the results
dfresult = pd.DataFrame(results)

# Display the result DataFrame
print(dfresult)
dfresultleft = dfresult

    emoji   code
0       😀   2050
1       😁   6659
2       😂  10256
3       😃   7184
4       😄  11796
..    ...    ...
208     🛜  30699
209     🛝   5103
210     🛞   8178
211     🛟   2548
212     🛡   3070

[213 rows x 2 columns]


In [67]:
import pandas as pd

# Assuming dfresultleft and dfresultlastcol are your DataFrames

# Step 1: Concatenate the DataFrames
combined_df = pd.concat([dfresultleft, dfresultlastcol], ignore_index=True)

# Step 2: Drop duplicate rows
unique_rows_df = combined_df.drop_duplicates()

# Display the resulting DataFrame with unique rows
print(unique_rows_df)

    emoji   code
0       😀   2050
1       😁   6659
2       😂  10256
3       😃   7184
4       😄  11796
..    ...    ...
209     🛝   5103
210     🛞   8178
211     🛟   2548
212     🛡   3070
263     🙼    284

[214 rows x 2 columns]


In [66]:
import pandas as pd

# Assuming dfresultleft and dfresultlastcol have the same structure
# Rename columns if necessary for clarity, e.g., 'value_left' and 'value_right'

# Merge the two DataFrames on the first column with an indicator
merged_df = pd.merge(dfresultleft, dfresultlastcol, on='emoji', suffixes=('_left', '_right'))

# Step 1: Filter for rows where the second column values are different
differing_rows = merged_df[merged_df['code_left'] != merged_df['code_right']]

# Step 2: Create a new DataFrame with the required format
result_df = differing_rows[['emoji', 'code_left', 'code_right']]

# Display the resulting DataFrame
print(result_df)

Empty DataFrame
Columns: [emoji, code_left, code_right]
Index: []


In [69]:
import pandas as pd
dfemoji = pd.read_csv("/Users/adityagautam/Downloads/mini-project-1/datasets/train/train_emoticon.csv")
# Assuming dfresult has columns 'emoji' and 'code', dfemoji has a column with a string of 13 emojis,
# and dfcode has 'input_str' (the corresponding ciphered number strings)

# Step 1: Create a dictionary mapping emojis to their corresponding code
emoji_to_code = dict(zip(unique_rows_df['emoji'], unique_rows_df['code']))

# Step 2: Define a function to generate the code string 
def generate_code_string(emoji_string):
    #last_5_emojis = emoji_string[-5:]  # Get the last 5 emojis
    code_string = ''.join([emoji_to_code.get(emoji, '') for emoji in emoji_string])  # Get codes for each emoji
    return code_string

# Step 3: Apply the function to dfemoji to generate the code string for each row
dfemoji['code_string'] = dfemoji.iloc[:, 0].apply(generate_code_string)
dfemoji

,input_emoticon,label,code_string
0,😛🛐😻😑😣🙠🙯🚼😒🙼😑🙯😣,0,1543646427181596614280026242231322841596262614
1,🛐😑😪😛🚼🙯😣🚅😑🙯😹😣🙼,0,46415963695154364222626141104715962624761614284
2,😛🙯😑🚡😣🚼🛐🙲😣🙯🛑😑🙼,0,15436262159659996144224641358061426240511596284
3,😛🚼🛐🙐😣🙯😑🙪😑🙼🛆😣🙯,1,1543642246412386142621596895615962843510614262
4,🛐🚟🚼😛🙋😑😣🙯😹🙯😑😣🙼,1,46418994221543620690159661426247612621596614284
...,...,...,...
7075,🙯😺😻😑😣🛐😛🚼🙼🙯😣😑🚠,1,262670271815966144641543642228426261415962414
7076,🛐🚼😅🙯😣🙹😛😑😑🙼😣🚍🙯,1,4644221558262614334915436159615962846143901262
7077,🛜😑🙒😛😣🙯🛐🚼🙼🙬🙯😣😑,1,30699159643091543661426246442228417952626141596
7078,😣😑😛🛋🛐🙯🚼🚪🙯😑🛀😣🙼,0,6141596154361983464262422377582621596940614284


In [77]:
# Calculate the length of each string in the 'code' column
string_lengths = dfemoji['code_string'].dropna().apply(len)

# Count the frequency of each string length
length_frequency = string_lengths.value_counts().sort_index()

# Display the frequency of string sizes
print(length_frequency)

50    7080
Name: code_string, dtype: int64


In [71]:
# Define a function to pad the code_string with zeros
def pad_code_string(code_string):
    if isinstance(code_string, str) and len(code_string) < 50:
        return code_string.zfill(50)  # Pad with zeros to the left to make length 50
    return code_string  # Return unchanged if not a string or length is already 50 or more

# Apply the padding function to the 'code_string' column
dfemoji['code_string'] = dfemoji['code_string'].apply(pad_code_string)

# Display the updated dfemoji dataframe
print(dfemoji)

     input_emoticon  label                                        code_string
0     😛🛐😻😑😣🙠🙯🚼😒🙼😑🙯😣      0  0000154364642718159661428002624223132284159626...
1     🛐😑😪😛🚼🙯😣🚅😑🙯😹😣🙼      0  0004641596369515436422262614110471596262476161...
2     😛🙯😑🚡😣🚼🛐🙲😣🙯🛑😑🙼      0  0001543626215965999614422464135806142624051159...
3     😛🚼🛐🙐😣🙯😑🙪😑🙼🛆😣🙯      1  0000154364224641238614262159689561596284351061...
4     🛐🚟🚼😛🙋😑😣🙯😹🙯😑😣🙼      1  0004641899422154362069015966142624761262159661...
...             ...    ...                                                ...
7075  🙯😺😻😑😣🛐😛🚼🙼🙯😣😑🚠      1  0000026267027181596614464154364222842626141596...
7076  🛐🚼😅🙯😣🙹😛😑😑🙼😣🚍🙯      1  0000464422155826261433491543615961596284614390...
7077  🛜😑🙒😛😣🙯🛐🚼🙼🙬🙯😣😑      1  0003069915964309154366142624644222841795262614...
7078  😣😑😛🛋🛐🙯🚼🚪🙯😑🛀😣🙼      0  0000614159615436198346426242237758262159694061...
7079  😛😣🚼😑🛐😹🙯🚭😑😣🚟🙯🙼      0  0000154366144221596464476126219231596614189926...

[7080 rows x 3 columns]


In [75]:
dfcode = pd.read_csv("/Users/adityagautam/Downloads/mini-project-1/datasets/train/train_text_seq.csv")

In [80]:
# Step 1: Create a new DataFrame to store the differing rows
differences = pd.DataFrame()


# Step 2: Compare the two columns
differences['dfemoji_code_string'] = dfemoji['code_string']
differences['dfcode_input_str'] = dfcode['input_str']

# Step 3: Filter for rows where the values are different
differences = differences[differences['dfemoji_code_string'] != differences['dfcode_input_str']]

# Display the differing rows
differences


,dfemoji_code_string,dfcode_input_str
209,0002529615436422464159626217316142621596336528...,0000529615436422464159626217316142621596336528...
509,0001795614262464159615436422252965066284614159...,0000179561426246415961543642252965066284614159...
521,0005066154366141596422252964642622846142621596...,0000506615436614159642252964642622846142621596...
545,0002624641596252961543661443044226144304159626...,0000262464159652961543661443044226144304159626...
960,0006141596252964644222623720154362622846141596...,0000614159652964644222623720154362622846141596...
1048,0004221433614262252961596154364642844310614159...,0000422143361426252961596154364642844310614159...
1144,0006141543642225296262464368215961596614182128...,0000614154364225296262464368215961596614182128...
1251,0002529627181543661442215962624642999614284262...,0000529627181543661442215962624642999614284262...
1409,0001495252964221596262464614154362626143559284...,0000149552964221596262464614154362626143559284...
1534,0004222624646142624252961596154362841596207826...,0000422262464614262452961596154362841596207826...


In [82]:
# one mistake found in code 25296. should be 5296
# Use .copy() to ensure that we are working on a copy of the DataFrame
unique_rows_df = unique_rows_df.copy()

# Safely update the 'code' column value from '25296' to '5296'
unique_rows_df.loc[unique_rows_df['code'] == '25296', 'code'] = '5296'

# Display the updated DataFrame to verify the change
print(unique_rows_df)

    emoji   code
0       😀   2050
1       😁   6659
2       😂  10256
3       😃   7184
4       😄  11796
..    ...    ...
209     🛝   5103
210     🛞   8178
211     🛟   2548
212     🛡   3070
263     🙼    284

[214 rows x 2 columns]


In [83]:
import pandas as pd
dfemoji = pd.read_csv("/Users/adityagautam/Downloads/mini-project-1/datasets/train/train_emoticon.csv")
# Assuming dfresult has columns 'emoji' and 'code', dfemoji has a column with a string of 13 emojis,
# and dfcode has 'input_str' (the corresponding ciphered number strings)

# Step 1: Create a dictionary mapping emojis to their corresponding code
emoji_to_code = dict(zip(unique_rows_df['emoji'], unique_rows_df['code']))

# Step 2: Define a function to generate the code string 
def generate_code_string(emoji_string):
    #last_5_emojis = emoji_string[-5:]  # Get the last 5 emojis
    code_string = ''.join([emoji_to_code.get(emoji, '') for emoji in emoji_string])  # Get codes for each emoji
    return code_string

# Step 3: Apply the function to dfemoji to generate the code string for each row
dfemoji['code_string'] = dfemoji.iloc[:, 0].apply(generate_code_string)

# Define a function to pad the code_string with zeros
def pad_code_string(code_string):
    if isinstance(code_string, str) and len(code_string) < 50:
        return code_string.zfill(50)  # Pad with zeros to the left to make length 50
    return code_string  # Return unchanged if not a string or length is already 50 or more

# Apply the padding function to the 'code_string' column
dfemoji['code_string'] = dfemoji['code_string'].apply(pad_code_string)

# Display the updated dfemoji dataframe
print(dfemoji)


     input_emoticon  label                                        code_string
0     😛🛐😻😑😣🙠🙯🚼😒🙼😑🙯😣      0  0000154364642718159661428002624223132284159626...
1     🛐😑😪😛🚼🙯😣🚅😑🙯😹😣🙼      0  0004641596369515436422262614110471596262476161...
2     😛🙯😑🚡😣🚼🛐🙲😣🙯🛑😑🙼      0  0001543626215965999614422464135806142624051159...
3     😛🚼🛐🙐😣🙯😑🙪😑🙼🛆😣🙯      1  0000154364224641238614262159689561596284351061...
4     🛐🚟🚼😛🙋😑😣🙯😹🙯😑😣🙼      1  0004641899422154362069015966142624761262159661...
...             ...    ...                                                ...
7075  🙯😺😻😑😣🛐😛🚼🙼🙯😣😑🚠      1  0000026267027181596614464154364222842626141596...
7076  🛐🚼😅🙯😣🙹😛😑😑🙼😣🚍🙯      1  0000464422155826261433491543615961596284614390...
7077  🛜😑🙒😛😣🙯🛐🚼🙼🙬🙯😣😑      1  0003069915964309154366142624644222841795262614...
7078  😣😑😛🛋🛐🙯🚼🚪🙯😑🛀😣🙼      0  0000614159615436198346426242237758262159694061...
7079  😛😣🚼😑🛐😹🙯🚭😑😣🚟🙯🙼      0  0000154366144221596464476126219231596614189926...

[7080 rows x 3 columns]


In [84]:
# Step 1: Create a new DataFrame to store the differing rows
differences = pd.DataFrame()


# Step 2: Compare the two columns
differences['dfemoji_code_string'] = dfemoji['code_string']
differences['dfcode_input_str'] = dfcode['input_str']

# Step 3: Filter for rows where the values are different
differences = differences[differences['dfemoji_code_string'] != differences['dfcode_input_str']]

# Display the differing rows
differences


,dfemoji_code_string,dfcode_input_str


In [85]:
# Convert unique_rows_df to a CSV file
unique_rows_df.to_csv('unique_rows_df.csv', index=False)

# If you want to save it to a specific path, specify the full path like this:
# unique_rows_df.to_csv('/path/to/your/directory/unique_rows_df.csv', index=False)

In [86]:
unique_rows_df

,emoji,code
0,😀,2050
1,😁,6659
2,😂,10256
3,😃,7184
4,😄,11796
...,...,...
209,🛝,5103
210,🛞,8178
211,🛟,2548
212,🛡,3070
